#연관키워드 찾는 코드

In [7]:
import json

# JSON 데이터 로드 함수
def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

# 데이터 로드
input_file_path = "data/cleaned_video_comments.json"
data = load_json(input_file_path)

# 모든 댓글 리스트 가져오기
all_comments = [comment for category in data.values() for video in category for comment in video["comments"]]



#Binary n-gram

In [8]:
from kiwipiepy import Kiwi
from nltk.util import ngrams
from collections import Counter
import json

# JSON 데이터 로드 함수
def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

# 형태소 분석기 객체 생성
kiwi = Kiwi()

# 2글자 이상 명사만 추출하는 함수
def extract_nouns(text):
    nouns = [word for word, tag, _, _ in kiwi.analyze(text)[0][0] if tag.startswith("NNG") and len(word) > 1]
    return nouns

# N-gram 추출 함수 (명사만 활용)
def get_noun_ngrams(texts, n=2):
    all_ngrams = []
    for text in texts:
        nouns = extract_nouns(text)  # 명사만 추출
        if len(nouns) >= n:  # N-gram을 만들기 위해 최소 n개 단어 필요
            all_ngrams.extend(ngrams(nouns, n))
    
    return Counter(all_ngrams)  # N-gram 빈도수 계산

# JSON 데이터 로드
input_file_path = "data/cleaned_video_comments.json"
data = load_json(input_file_path)

# 모든 댓글 리스트 가져오기
all_comments = [comment for category in data.values() for video in category for comment in video["comments"]]

# Bigram(2-gram) 분석 실행 (명사만 사용)
bigram_counter = get_noun_ngrams(all_comments, n=2)

# 상위 10개 연관 키워드 출력
print("📌 두 글자 이상 명사만 포함된 상위 10개 연관 키워드:")
for ngram, freq in bigram_counter.most_common(10):
    print(f"{ngram} -> {freq}회 등장")


📌 두 글자 이상 명사만 포함된 상위 10개 연관 키워드:
('부정', '선거') -> 619회 등장
('음주', '운전') -> 460회 등장
('영상', '감사') -> 432회 등장
('정보', '감사') -> 407회 등장
('탄핵', '반대') -> 322회 등장
('고인', '명복') -> 321회 등장
('자유', '민주주의') -> 210회 등장
('대통령', '탄핵') -> 186회 등장
('탄핵', '무효') -> 160회 등장
('국회', '의원') -> 157회 등장


#Tri n-gram

In [9]:
from kiwipiepy import Kiwi
from nltk.util import ngrams
from collections import Counter
import json

# JSON 데이터 로드 함수
def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

# 형태소 분석기 객체 생성
kiwi = Kiwi()

# 2글자 이상 명사만 추출하는 함수
def extract_nouns(text):
    nouns = [word for word, tag, _, _ in kiwi.analyze(text)[0][0] if tag.startswith("NNG") and len(word) > 1]
    return nouns

# N-gram 추출 함수 (명사만 활용)
def get_noun_ngrams(texts, n=2):
    all_ngrams = []
    for text in texts:
        nouns = extract_nouns(text)  # 명사만 추출
        if len(nouns) >= n:  # N-gram을 만들기 위해 최소 n개 단어 필요
            all_ngrams.extend(ngrams(nouns, n))
    
    return Counter(all_ngrams)  # N-gram 빈도수 계산

# JSON 데이터 로드
input_file_path = "data/cleaned_video_comments.json"
data = load_json(input_file_path)

# 모든 댓글 리스트 가져오기
all_comments = [comment for category in data.values() for video in category for comment in video["comments"]]

trigram_counter = get_noun_ngrams(all_comments, n=3)

# 상위 10개 연관 키워드 출력
print("📌 두 글자 이상 명사만 포함된 상위 10개 연관 키워드:")
for ngram, freq in bigram_counter.most_common(10):
    print(f"{ngram} -> {freq}회 등장")


📌 두 글자 이상 명사만 포함된 상위 10개 연관 키워드:
('부정', '선거') -> 619회 등장
('음주', '운전') -> 460회 등장
('영상', '감사') -> 432회 등장
('정보', '감사') -> 407회 등장
('탄핵', '반대') -> 322회 등장
('고인', '명복') -> 321회 등장
('자유', '민주주의') -> 210회 등장
('대통령', '탄핵') -> 186회 등장
('탄핵', '무효') -> 160회 등장
('국회', '의원') -> 157회 등장


연관키워드 추출은 잘 되는거 같아. 그러면 이제 목표를 조금 바꿀게.
final_keywords.json에 키워드랭킹이 나오잖아. 
이제 그 랭킹에 있는 각 키워드들의 연관키워드를 추출 해내고 싶어.
위에서 작동한 방식대로 코딩을 해서 특정 키워드의 연관키워드를 추출 해낼 수 있어?

#특정키워드의 연관 키워드 추출

In [20]:
import json
from nltk.util import ngrams
from collections import Counter

# JSON 데이터 로드 함수
def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

# N-gram 추출 함수 (이미 명사만 저장된 데이터 사용)
def get_noun_ngrams(nouns_data, n=2):
    all_ngrams = []
    for category in nouns_data.values():
        for video in category:
            for nouns in video["nouns"]:  # 댓글별 명사 리스트 사용
                if len(nouns) >= n:  # 최소 n개 단어 필요
                    all_ngrams.extend(ngrams(nouns, n))
    
    return Counter(all_ngrams)  # N-gram 빈도수 계산

# 특정 키워드와 연관된 N-gram 찾기
def find_related_keywords(ngrams_counter, target_word):
    related_words = []
    for ngram, freq in ngrams_counter.items():
        if target_word in ngram:  # 특정 키워드가 포함된 N-gram 찾기
            related_words.append((ngram, freq))

    return sorted(related_words, key=lambda x: x[1], reverse=True)  # 빈도순 정렬

# JSON 데이터 로드
input_file_path = "data/kiwi_nouns_video_comments.json"
data = load_json(input_file_path)

# Bigram(2-gram) 분석 실행 (명사만 사용)
bigram_counter = get_noun_ngrams(data, n=2)

# 🔥 특정 키워드 입력 후 연관 키워드 찾기
target_word = "로블록스"  # 예제 키워드
related_keywords = find_related_keywords(bigram_counter, target_word)

# 결과 출력
print(f"📌 '{target_word}'와 함께 자주 등장하는 연관 키워드:")
for ngram, freq in related_keywords[:10]:  # 상위 10개 출력
    print(f"{ngram} -> {freq}회 등장")


📌 '로블록스'와 함께 자주 등장하는 연관 키워드:
('로블록스', '유튜버') -> 15회 등장
('유튜버', '로블록스') -> 12회 등장
('괴물', '로블록스') -> 11회 등장
('로블록스', '시리즈') -> 4회 등장
('로블록스', '게임') -> 4회 등장
('로블록스', '피쉬') -> 2회 등장
('로블록스', '공포') -> 1회 등장
('겜급동', '로블록스') -> 1회 등장
('로블록스', '행복') -> 1회 등장
('로블록스', '차세대') -> 1회 등장
